In [1]:
import time

In [2]:
from juggling_dlx_milp import *

/home/josue/Documents/git/Jugglyng1/Expérimentations/Juggling DLX/DLX/dlxm.py:69: RuntimeWarning: class "DLX_M::AbstrItem" has no virtual destructor
  class ConcItem(_AbstrItem):


### Musique à jouer

In [3]:
music = [( 1, "do"), ( 2, "do"), ( 3, "do"), 
         ( 4, "re"), ( 5, "mi"), ( 7, "re"), 
         ( 9, "do"), (10, "mi"), (11, "re"),
         (12, "re"), (13, "do")]

In [4]:
music = [(1, "re5"), (2, "do5"), (3, "re5"), (4, "do5"),
         (5, "re5"), (7, "do5"), (9, "mi5"),
         (11, "re5"), (12, "fa5"), (13, "la5"), (14, "re6"), (15, "fa6")]

### Contraintes

In [5]:
nb_hands = 4
max_height = 5
max_weight = 2
forbidden_multiplex = [(1, 2), (1, 3), (1, 4), (2, )]
multiple_throws = True

### Algorithme

In [6]:
balls, throws = music_to_throws(music)

In [7]:
ec_instance = throws_to_extended_exact_cover(balls, throws, nb_hands, max_height, max_weight,
                                             forbidden_multiplex, [], multiple_throws)

In [ ]:
t = time.time()
sols = solve_with_dlx(ec_instance)
print("{} solutions trouvées en {} secondes".format(len(sols), time.time() - t))

In [17]:
print_juggling(sols[0])

{'do5', 'mi5', 're5', 'fa5'}       {}       : re5 -- 1 --> 1
   {'do5', 'mi5', 'fa5'}        {'re5'}     : do5 -- 1 --> 1
       {'mi5', 'fa5'}        {'do5', 're5'} : re5 -- 1 --> 0
   {'mi5', 'fa5', 're5'}        {'do5'}     : do5 -- 1 --> 0
{'do5', 'mi5', 'fa5', 're5'}       {}       : re5 -- 1 --> 1
   {'do5', 'mi5', 'fa5'}        {'re5'}     : 
   {'do5', 'mi5', 'fa5'}        {'re5'}     : do5 -- 1 --> ?
       {'mi5', 'fa5'}           {'re5'}     : 
       {'mi5', 'fa5'}           {'re5'}     : mi5 -- 1 --> ?
          {'fa5'}               {'re5'}     : 
          {'fa5'}               {'re5'}     : re5 -- 1 --> ?
          {'fa5'}                  {}       : fa5 -- 1 --> ?


In [18]:
import modele
import ipywidgets
import pythreejs

balls = [{"color":"blue", "tone":"do5", "name":"do5"},
         {"color":"red", "tone":"re5", "name":"re5"},
         {"color":"green", "tone":"mi5", "name":"mi5"},
         {"color":"yellow", "tone":"fa5", "name":"fa5"},
         {"color":"yellow", "tone":"la5", "name":"la5"},
         {"color":"yellow", "tone":"re6", "name":"re5"},
         {"color":"yellow", "tone":"fa6", "name":"fa6"}]

# balls = [{"color":"blue", "tone":"do", "name":"do"},
#          {"color":"red", "tone":"re", "name":"re"},
#          {"color":"green", "tone":"mi", "name":"mi"}]

pattern = juggling_sol_to_simulator(sols[0])

sides = [-1, 1]

model = modele.Model(balls, pattern)
slider = ipywidgets.FloatSlider(min=0, max=40, step=0.1)
view = modele.View(model, sides)
slider.observe(lambda change: view.update(change['new'], change['old']), names="value")
ipywidgets.VBox([view.widget,slider])